<h1>Trabalho de Modelagem e Simulação</h1>
<p style='font-size: 20px'>Aluno: Artur Bernardo Mallmann</p>
Importa classe de

In [1]:
from abc import ABCMeta, abstractmethod, abstractproperty
import pandas
# class IGen:
#     __metaclass__ = ABCMeta

<h2>Classe para ler CSVs</h2>

In [2]:
from projeto import MyCSVs
arquivo = MyCSVs('entradas.csv')
help(MyCSVs)


# arquivo = pandas.read_csv('entradas.csv')
arquivo.print_html()
# print(arquivo.get_item('ir',0))

Help on class MyCSVs in module projeto:

class MyCSVs(builtins.object)
 |  MyCSVs(file_name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file_name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __next__(self)
 |  
 |  get_col(self, col)
 |  
 |  get_dict(self)
 |  
 |  get_line(self, line, campos=None)
 |  
 |  get_names(self)
 |  
 |  print_html(self)
 |  
 |  where(self, campo, valor, campos=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



ir,fichar,consultar,pagar
3,7,23,4
5,11,15,4
8,8,18,2
9,7,17,3
11,12,22,2
12,10,25,4
14,9,21,3
17,8,22,4
18,10,16,3
21,7,20,2


<h2>Gerar Valores por monte Carlo</h2>
Caso queiramos gerar valores baseado em tempos de chegada amostrados em um CSV, basta passá-lo por por parâmetro

In [3]:
from montecarlo import MonteCarlo
classes = 8 # tipos de amostras diferentes

In [4]:
#mcs é objeto de uma das classes que gera amostras, que tbm pode ser geradores aleatórios quaisquer
mcs = dict()
for en in arquivo.get_names():
    mcs.update({en:MonteCarlo(classes,arquivo.get_col(en))})
# for en in arquivo.keys():
#     mcs.update({en:MonteCarlo(classes,arquivo[en])})

2.25
classes: 8
delta: 2.25
increment: [0, 0, 2, 2, 3, 4, 4, 6, 6, 7]
freq:[0.2, 0.2, 0.4, 0.5, 0.7, 0.7, 0.8999999999999999, 0.9999999999999999]
pms:[4.125, 6.375, 8.625, 10.875, 13.125, 15.375, 17.625, 19.875]
0.625
classes: 8
delta: 0.625
increment: [0, 6, 1, 0, 7, 4, 3, 1, 4, 0]
freq:[0.3, 0.5, 0.5, 0.6, 0.8, 0.8, 0.9, 1.0]
pms:[7.3125, 7.9375, 8.5625, 9.1875, 9.8125, 10.4375, 11.0625, 11.6875]
1.25
classes: 8
delta: 1.25
increment: [6, 0, 2, 1, 5, 7, 4, 5, 0, 4]
freq:[0.2, 0.30000000000000004, 0.4, 0.4, 0.6000000000000001, 0.8, 0.9, 1.0]
pms:[15.625, 16.875, 18.125, 19.375, 20.625, 21.875, 23.125, 24.375]
0.25
classes: 8
delta: 0.25
increment: [7, 7, 0, 4, 0, 7, 4, 7, 4, 0]
freq:[0.3, 0.3, 0.3, 0.3, 0.6, 0.6, 0.6, 1.0]
pms:[2.125, 2.375, 2.625, 2.875, 3.125, 3.375, 3.625, 3.875]


In [5]:
mc = MonteCarlo(10,[1,2,3,10])
# MonteCarlo.__str__ = lambda self:f'{self.get_rand()}'

0.9
classes: 10
delta: 0.9
increment: [0, 1, 2, 9]
freq:[0.25, 0.5, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 1.0]
pms:[1.45, 2.35, 3.25, 4.15, 5.05, 5.95, 6.8500000000000005, 7.75, 8.649999999999999, 9.549999999999999]


In [6]:
print(mcs['ir'].get_rand())

17.625


In [7]:
print(arquivo.get_names())
# print(arquivo.keys())

['ir', 'fichar', 'consultar', 'pagar']


<h2>Replay: Ressimular baseado em um CSV</h2>
Caso queira rever uma simulação específica basta passar o CSV por parâmetro.

In [8]:
from projeto import IGen

class Replay(IGen):
    def __init__(self,inputs):
        self.inputs = list(inputs)
    def get_rand(self):
        return self.inputs.pop(0) if len(self.inputs)!=0 else -1 #sempre retira o primeiro

In [9]:
# file_name = 'entradas.csv'
classes = 8
replay = dict()
for en in arquivo.get_names():
    col = arquivo.get_col(en)
    replay.update({en:Replay(arquivo.get_col(en))})
# for en in arquivo.keys():
#     replay.update({en:Replay(arquivo[en])})

In [10]:
print(replay['ir'].get_rand())

3


<h2>Rand: Entradas randômicas</h2>
Gera os números de entrada de forma randômica.

In [11]:
import random
class GenRand(IGen):
    def __init__(self,mini,maxi):
        self.bounds=(mini,maxi)
    def get_rand(self):
        return random.uniform(*self.bounds)
aaa = GenRand(2,5)
print(aaa.get_rand())

3.4312814119007697


<h1>Simulador SimFlex</h1>
O simulador <b>SimFlex</b> tem o intuito de ser uma ferramenta totalmente flexível e configurável pelo usuário.
Os arquivos de configurações descrevem totalmente o funcionamento da simulação. Eles são:
entidades.csv, operacoes.csv e entidades.csv(opcional)

<b>entidades.csv:</b> Arquivo responsável por configurar a alocação de Recursos e dados como tempo de execução em cada processo lógico.

In [12]:
entidades = MyCSVs('entidades.csv')
entidades.print_html()
# entidades = pandas.read_csv('entidades.csv')
# entidades

LP,entidades,acoes,tmin,tmax
fila,0,ir,2,5
balcao,2,fichar,7,12
balcao,2,pagar,2,4
clinica,3,consultar,15,25


<b>operacoes.csv:</b> Este arquivo é responsável por descrever qual o fluxo das entidades dentro do sistema

In [13]:
operacoes = MyCSVs('operacoes.csv')
operacoes.print_html()
# operacoes = pandas.read_csv('operacoes.csv')
# operacoes

stage,value,dest,nvalue
fila,1,balcao,2
balcao,2,consulta,3
balcao,4,saida,4
clinica,3,balcao,5


<b>entradas.csv</b>(opcional)<b>:</b> Arquivo com a descrição de tempos de eventos pré definidos para gerar novos tempos por monte carlo ou para reproduzir um replay.

In [14]:
entradas = MyCSVs('entradas.csv')
entradas.print_html()
# entradas = pandas.read_csv('entradas.csv')
# entradas

ir,fichar,consultar,pagar
3,7,23,4
5,11,15,4
8,8,18,2
9,7,17,3
11,12,22,2
12,10,25,4
14,9,21,3
17,8,22,4
18,10,16,3
21,7,20,2


For each funcionando em todos:

In [15]:
for i in entidades:
    print(i)
for i in entradas:
    print (i)
for i in operacoes:
    print (i)
    
operacoes.where('stage','balcao')

('fila', 0, 'ir', 2, 5)
('balcao', 2, 'fichar', 7, 12)
('balcao', 2, 'pagar', 2, 4)
('clinica', 3, 'consultar', 15, 25)
(3, 7, 23, 4)
(5, 11, 15, 4)
(8, 8, 18, 2)
(9, 7, 17, 3)
(11, 12, 22, 2)
(12, 10, 25, 4)
(14, 9, 21, 3)
(17, 8, 22, 4)
(18, 10, 16, 3)
(21, 7, 20, 2)
('fila', 1, 'balcao', 2)
('balcao', 2, 'consulta', 3)
('balcao', 4, 'saida', 4)
('clinica', 3, 'balcao', 5)


[(2, 'consulta', 3), (4, 'saida', 4)]

In [16]:
# Implementacao assincrona baseada em: ref Chandy, K. M., & Misra, J. (1981). Asynchronous distributed simulation via a sequence of parallel computations. Communications of the ACM, 24(4), 198–206. doi:10.1145/358598.358613 


# class Pipe(self):

# class MsgStatus(enum):
#     FILA = 1
#     BALCAO = 2
#     CONSULTA = 3
#     SBALCAO = 4
#     SAIDA = 5

class Msg:
    def __init__(self,time,action):
        self.time=time
        self.action=action
    
# class Spv: #spv service providers (atendentes) ou pode ser usado como fila quando sem entrada.
#     def __init__(self,time):
#         self.time = time
#     def atende(self):
#         pass

class LogicalProcess:
    def __init__(self,lp,acoes,serv): 
        self.fila=[] # A princípio filas únicas
        self.serv=serv
        self.timers=dict()
        self.lvc = 0 #relógio local
        self.spvs = spvs
    def set_timer(action,timer):
        self.timers.update(action,timer)
    def qeue(self,msg):
        if (self.lvc <= msg.time):
            fila.append(msg)
    def execute(self):
        pass
    def notify(self,msg):
        pass

In [17]:
class Simulation:
    def __init__ (self,entidades,operacoes,entradas=None,replay=False):
        self.lps=dict()
        for entidade in entidades:
            LP,serv,action,tmin,tmax = entidade
            if(entradas == None):
                lp.set_time(action)
            lp = LogicalProcess()
            self.lps.update({entidade[0]:[]})
        for k in self.lps.keys():
            self.lps[k]=entidades.where('LP',k)
            
        self.acoes=dict()
        for k in self.lps.keys():
            self.acoes[k]=operacoes.where('stage',k)
        
        print(self.lps)
        print(self.acoes)
        self.temporizadores=[]
        self.tempo=0
#         print (self.entidades['LD'])
#         if (entradas == None):
#             pass
#     def criar_entidades()
#         pass

In [19]:
# csvs: entidades,operacoes,entradas:

In [20]:
sim = Simulation(entidades,operacoes)

{'fila': [(0, 'ir', 2, 5)], 'balcao': [(2, 'fichar', 7, 12), (2, 'pagar', 2, 4)], 'clinica': [(3, 'consultar', 15, 25)]}
{'fila': [(1, 'balcao', 2)], 'balcao': [(2, 'consulta', 3), (4, 'saida', 4)], 'clinica': [(3, 'balcao', 5)]}


In [ ]:
# file_name = 'entradas.csv'
# classes = 8
# # Arrumar csv errado:
# #lendo do csv erradão
# myf = open('entradas_old.csv','r')
# linhas = myf.readlines()
# linhas = [l.replace('\n','') for l in linhas if len(l)>1]
# myf.close()
# amostras=[]
# for linha in linhas:
#     amostras.append( [x for x in linha.split(',')] )
# print(amostras)

# lines = []
# lines.append('chegada, ficha, atendimento, pagamento')
# for i in zip(*amostras):
#     lines.append(','.join(i))
# print (lines)
# #     pass
# myf = open(file_name,'w')
# myf.writelines([line+'\n' for line in lines])
# myf.close()